### Display live camera feed

In [ ]:
#importing the libraries

import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

#instantiating the camera of 224x224 as the neural network takes this as an input
camera = Camera.instance(width=224, height=224)

# this width and height doesn't necessarily have to match the camera
image = widgets.Image(format='jpeg', width=224, height=224)  

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

We'll create a folder ``dataset`` that will contain two sub-folders ``free`` and ``blocked``, 
where we'll place the images for each scenario.

In [ ]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Now create and display some buttons that we'll use to save snapshots for each class label and also add some text boxes that will display how many images of each category that we've collected so far.

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

We have to attach functions to save images for each category to the buttons' ``on_click`` event.

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Great! Now the buttons above should save images to the ``free`` and ``blocked`` directories.

Collect some data by the following - 

1. Place the robot in a scenario where it's blocked and press ``add blocked``
2. Place the robot in a scenario where it's free and press ``add free``


Here are some tips for labeling data

1. Try different orientations
2. Try different lighting

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.

I approximately collected 600 images for each class.

In [ ]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Again, let's close the camera conneciton properly so that we can use the camera in the later notebook.

In [ ]:
camera.stop()